<a href="https://colab.research.google.com/github/Krieg2065/Python/blob/main/da_indirizzi_a_cordinate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import numpy as np
import shapely.geometry
import pandas as pd
import pymongo
import geopandas


locator = Nominatim()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
tuo_dataframe['location'] = tuo_dataframe['la Tua Colonna Con Vie QUI'].apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
tuo_dataframe['point'] = tuo_dataframe['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - fill all none values with NaN
tuo_dataframe.point.fillna(value=np.nan, inplace=True)
#5 - save all NaN values in another dataframe
tuo_dataframe_nan = tuo_dataframe[tuo_dataframe['point'].isnull()]
#6- drop all NaN values
tuo_dataframe.dropna(subset=['point'], inplace = True)
#7 - create a Longitude column from point column
tuo_dataframe['Longitude'] = tuo_dataframe.point.apply(lambda x: x[1])
#8 - create a Latitude column from point column
tuo_dataframe['Latitude'] = tuo_dataframe.point.apply(lambda x: x[0])
# 9 - convert DataFrane to GeoDataFrame
gdf = geopandas.GeoDataFrame(tuo_dataframe, geometry=geopandas.points_from_xy(tuo_dataframe.Longitude, tuo_dataframe.Latitude))
# 10 - drop all columns that are not needed
gdf.drop(['_id','point','Longitude','Latitude','location'], axis= 1,inplace = True) #drop id solo se hai la colonna _id di mongodb
# 11 - converting geographical data in a GeoPandas GeoDataframe to a MongoDB Geopoint object
gdf['geometry']=gdf['geometry'].apply(lambda x:shapely.geometry.mapping(x))
# 12 - inserting the GeoDataFrame in a collection on MongoDB
db.TuaCollection.insert_many(gdf.to_dict('records'))

if not tuo_dataframe_nan.empty:
  db.TuaCollectionNan.insert_many(tuo_dataframe_nan.to_dict('records')) #serve a salvare tutti i valori che non hai trovato sul database